# Chapter 7.2: Hypothetical Document Embeddings (HyDE)

이 노트북은 OSTEP RAG 시스템에 HyDE 기법을 적용하여 검색 품질을 개선하는 방법을 실습합니다.

## 📚 학습 목표
- HyDE 개념 이해: 쿼리를 가상의 답변 문서로 확장
- 짧은 쿼리와 긴 문서 간의 의미적 거리 문제 해결
- Ollama를 활용한 로컬 LLM 기반 HyDE 구현

## 📋 실습 구성
1) 설정 및 하이퍼파라미터 정의
2) 기존 데이터 및 Ollama 설정
3) HyDE 개념 설명
4) 가상 문서 생성 함수 구현
5) 비교 실험: 일반 검색 vs HyDE 검색
6) 다양한 쿼리로 HyDE 효과 확인

> **핵심 아이디어**: 짧은 쿼리 대신 "답변을 포함하는 가상 문서"를 생성하여 검색하면, 실제 문서와의 유사도가 높아져 검색 정확도가 향상됩니다.


---
## 1️⃣ 설정 및 하이퍼파라미터 정의

이 셀에서는 경로, 모델, 검색 파라미터 등을 정의합니다.

**주요 내용:**
- 기존 산출물 경로 (청크 JSON, FAISS 인덱스)
- Ollama 서버 설정 (로컬 LLM)
- HyDE 관련 파라미터 (생성할 가상 문서 길이 등)

**실행 결과:**
- 설정값이 출력됩니다.


In [ ]:
# ========================================
# 1️⃣ 설정 / 하이퍼파라미터
# ========================================
import os
import json
import faiss
import numpy as np
import torch
import requests
from pathlib import Path
from typing import List, Dict, Any

# 재현성
SEED = 42
np.random.seed(SEED)

# 디바이스
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# 경로 (기존 산출물)
CHUNK_FILE = "/home/kbs1102/workspace/OSTEP_RAG/data/chunk/ostep_tok400_ov20.json"
INDEX_FILE = "/home/kbs1102/workspace/OSTEP_RAG/data/index/ostep_hnsw.index"

# 검색 하이퍼파라미터
TOP_K = 5
EF_SEARCH = 64

# 임베딩 모델
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
NORMALIZE = True

# Ollama 설정 (HyDE용 LLM)
OLLAMA_HOST = "http://localhost:11434"
LLM_MODEL = "llama3.1:8b"
TEMPERATURE = 0.3  # 낮은 temperature로 더 일관된 가상 문서 생성
MAX_TOKENS = 300   # 가상 문서 길이

print(f"🔧 Device: {DEVICE}")
print(f"📁 청크 파일: {CHUNK_FILE}")
print(f"📁 인덱스 파일: {INDEX_FILE}")
print(f"🔎 TOP_K={TOP_K}, efSearch={EF_SEARCH}")
print(f"🧠 임베딩 모델: {EMBED_MODEL}")
print(f"🤖 LLM: {LLM_MODEL} @ {OLLAMA_HOST}")


---
## 2️⃣ 기존 데이터 및 Ollama 설정

Chapter 2-4에서 생성한 청크 JSON과 FAISS 인덱스를 로드하고, Ollama 서버 연결을 확인합니다.

**주요 내용:**
- 청크 데이터 로드
- FAISS 인덱스 로드
- 임베딩 모델 로드
- Ollama 서버 연결 확인

**실행 결과:**
- 데이터 로드 완료 메시지와 Ollama 서버 상태가 출력됩니다.


In [ ]:
# 청크 로드
print("청크 데이터 로드 중...")
with open(CHUNK_FILE, 'r', encoding='utf-8') as f:
    CHUNKS = json.load(f)
print(f"✓ 청크 로드 완료: {len(CHUNKS)}개")

# 인덱스 로드
print("\n인덱스 로드 중...")
INDEX = faiss.read_index(INDEX_FILE)
print(f"✓ 인덱스 로드 완료: {INDEX.ntotal}개 벡터")

# 임베딩 모델 로드
print(f"\n임베딩 모델 로드 중: {EMBED_MODEL}")
from sentence_transformers import SentenceTransformer
embed_model = SentenceTransformer(EMBED_MODEL, device="cpu" if DEVICE=="cpu" else DEVICE)
print("✓ 임베딩 모델 로드 완료")

# Ollama 서버 연결 확인
print(f"\nOllama 서버 연결 확인: {OLLAMA_HOST}")
try:
    response = requests.get(f"{OLLAMA_HOST}/api/version", timeout=5)
    if response.status_code == 200:
        print(f"✓ Ollama 서버 연결 성공: {response.json()}")
    else:
        print(f"⚠️  Ollama 서버 응답 오류: {response.status_code}")
except Exception as e:
    print(f"⚠️  Ollama 서버 연결 실패: {e}")
    print("   Ollama 서버가 실행 중인지 확인하세요: ollama serve")


---
## 3️⃣ HyDE 개념 설명

이 셀에서는 HyDE의 핵심 개념을 설명합니다.

**HyDE란?**
- **문제**: 짧은 쿼리 vs 긴 문서 사이의 의미적 거리가 큼
  - 예: 쿼리 "virtual memory" vs 문서 "Virtual memory is a memory management technique..."
- **해결책**: 쿼리를 "답변을 포함하는 가상 문서"로 확장
  1. 쿼리를 받음: "What is virtual memory?"
  2. LLM이 가상 답변 문서 생성: "Virtual memory is a technique that..."
  3. 가상 문서를 임베딩하여 검색
- **효과**: 가상 문서와 실제 문서가 더 유사한 형식이므로 검색 정확도 향상

**HyDE의 장점:**
- 쿼리-문서 간 형식 차이 해소
- 복잡한 질문의 의도를 더 잘 포착
- 다양한 표현 방식을 하나의 가상 문서로 통합

**실행 결과:**
- 개념 설명이 출력됩니다.


In [ ]:
print("=" * 80)
print("[HyDE (Hypothetical Document Embeddings) 개념]")
print("=" * 80)
print()
print("전통적 검색:")
print("  쿼리: 'What is virtual memory?' → 임베딩 → 검색")
print()
print("HyDE 검색:")
print("  쿼리: 'What is virtual memory?'")
print("  ↓ LLM 생성")
print("  가상 문서: 'Virtual memory is a memory management technique...'")
print("  ↓ 임베딩")
print("  검색 (실제 문서와 형식이 유사하여 더 정확한 매칭)")
print()
print("=" * 80)
print("💡 핵심: 짧은 질문보다 긴 답변 형식이 문서와 더 잘 매칭됩니다.")
print("=" * 80)


---
## 4️⃣ 가상 문서 생성 함수 구현

이 셀에서는 Ollama를 사용하여 쿼리로부터 가상 문서를 생성하는 함수를 구현합니다.

**주요 내용:**
- Ollama API 호출 함수
- 가상 문서 생성 프롬프트 설계
- 샘플 쿼리로 가상 문서 생성 테스트

**실행 결과:**
- 샘플 쿼리에 대한 가상 문서가 생성되어 출력됩니다.


In [ ]:
def ollama_generate(prompt: str, temperature: float = TEMPERATURE, max_tokens: int = MAX_TOKENS) -> str:
    """
    Ollama를 사용하여 텍스트를 생성합니다.
    
    Args:
        prompt: 생성할 프롬프트
        temperature: 생성 temperature
        max_tokens: 최대 토큰 수
    
    Returns:
        생성된 텍스트
    """
    payload = {
        "model": LLM_MODEL,
        "prompt": prompt,
        "stream": False,
        "options": {
            "temperature": temperature,
            "num_predict": max_tokens,
        }
    }
    try:
        r = requests.post(f"{OLLAMA_HOST}/api/generate", json=payload, timeout=60)
        r.raise_for_status()
        data = r.json()
        return data.get("response", "")
    except Exception as e:
        return f"[Error] {e}"

def generate_hypothetical_document(query: str) -> str:
    """
    쿼리로부터 가상 문서를 생성합니다.
    
    Args:
        query: 사용자 질문
    
    Returns:
        가상 답변 문서
    """
    prompt = f"""Given the question below, write a detailed and informative passage that answers the question. 
The passage should be similar in style to an educational textbook or technical documentation.
Do not include the question in your response, only provide the answer passage.

Question: {query}

Answer passage:"""
    
    hypothetical_doc = ollama_generate(prompt)
    return hypothetical_doc.strip()

# 샘플 테스트
print("=" * 80)
print("[가상 문서 생성 테스트]")
print("=" * 80)

test_query = "How does the operating system handle virtual memory?"
print(f"\n쿼리: {test_query}")
print("\n가상 문서 생성 중...")
hypothetical_doc = generate_hypothetical_document(test_query)
print(f"\n생성된 가상 문서:")
print("-" * 80)
print(hypothetical_doc)
print("-" * 80)
print(f"\n가상 문서 길이: {len(hypothetical_doc)} 글자")


---
## 5️⃣ 비교 실험: 일반 검색 vs HyDE 검색

이 셀에서는 동일한 쿼리에 대해 일반 검색과 HyDE 검색을 비교합니다.

**주요 내용:**
- 일반 검색: 쿼리를 직접 임베딩하여 검색
- HyDE 검색: 가상 문서를 생성하고 임베딩하여 검색
- 결과 비교

**실행 결과:**
- 두 방식의 검색 결과와 유사도 점수가 출력됩니다.


In [ ]:
def distance_to_similarity(distances: np.ndarray) -> np.ndarray:
    """L2 거리를 유사도로 변환 (정규화된 벡터 가정)"""
    return 1.0 - (distances / 2.0)

def search_with_text(text: str, k: int = TOP_K):
    """텍스트를 임베딩하여 FAISS 인덱스에서 검색"""
    faiss.ParameterSpace().set_index_parameter(INDEX, "efSearch", EF_SEARCH)
    text_emb = embed_model.encode([text], convert_to_numpy=True, normalize_embeddings=NORMALIZE).astype('float32')
    D, I = INDEX.search(text_emb, k)
    scores = distance_to_similarity(D[0])
    return I[0], scores

# 테스트 쿼리
TEST_QUERY = "How does the operating system manage virtual memory?"

print("=" * 80)
print(f"테스트 쿼리: {TEST_QUERY}")
print("=" * 80)

# 1) 일반 검색
print("\n[1. 일반 검색 (쿼리 직접 사용)]")
print("-" * 80)
indices_normal, scores_normal = search_with_text(TEST_QUERY, k=3)
for rank, (idx, score) in enumerate(zip(indices_normal, scores_normal), 1):
    if idx >= 0:
        chunk = CHUNKS[idx]
        print(f"\n{rank}. Score: {score:.4f}")
        print(f"   Chapter: {chunk.get('chapter_title', 'N/A')}")
        print(f"   Preview: {chunk['text'][:120]}...")

# 2) HyDE 검색
print("\n" + "=" * 80)
print("[2. HyDE 검색 (가상 문서 사용)]")
print("-" * 80)
print("가상 문서 생성 중...")
hyde_doc = generate_hypothetical_document(TEST_QUERY)
print(f"✓ 가상 문서 생성 완료 ({len(hyde_doc)} 글자)")
print(f"\n가상 문서 미리보기:\n{hyde_doc[:200]}...\n")

indices_hyde, scores_hyde = search_with_text(hyde_doc, k=3)
for rank, (idx, score) in enumerate(zip(indices_hyde, scores_hyde), 1):
    if idx >= 0:
        chunk = CHUNKS[idx]
        print(f"\n{rank}. Score: {score:.4f}")
        print(f"   Chapter: {chunk.get('chapter_title', 'N/A')}")
        print(f"   Preview: {chunk['text'][:120]}...")

# 비교 결과
print("\n" + "=" * 80)
print("[비교 결과]")
print("=" * 80)
print(f"일반 검색 Top-1 점수: {scores_normal[0]:.4f}")
print(f"HyDE 검색 Top-1 점수: {scores_hyde[0]:.4f}")
print(f"\n개선도: {(scores_hyde[0] - scores_normal[0]):.4f}")
print("\n💡 HyDE는 가상 문서를 통해 쿼리를 더 풍부하게 표현하여")
print("   실제 문서와의 유사도를 높입니다.")


---
## 6️⃣ 다양한 쿼리로 HyDE 효과 확인

이 셀에서는 여러 쿼리로 HyDE의 효과를 추가 확인합니다.

**주요 내용:**
- 다양한 유형의 쿼리 테스트
- 일반 검색 vs HyDE 검색 비교
- 검색 결과 차이 분석

**실행 결과:**
- 각 쿼리별 검색 결과와 점수 개선도가 출력됩니다.


In [ ]:
# 추가 테스트 쿼리
test_queries = [
    "What is the role of the scheduler?",
    "Explain paging in memory management",
    "How do processes synchronize?"
]

print("=" * 80)
print("[추가 쿼리 테스트]")
print("=" * 80)

results = []

for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*80}")
    print(f"쿼리 {i}: {query}")
    print(f"{'='*80}")
    
    # 일반 검색
    indices_normal, scores_normal = search_with_text(query, k=1)
    
    # HyDE 검색
    print("가상 문서 생성 중...")
    hyde_doc = generate_hypothetical_document(query)
    indices_hyde, scores_hyde = search_with_text(hyde_doc, k=1)
    
    print(f"\n일반 검색 Top-1:")
    print(f"  Score: {scores_normal[0]:.4f}")
    print(f"  Chapter: {CHUNKS[indices_normal[0]].get('chapter_title', 'N/A')}")
    
    print(f"\nHyDE 검색 Top-1:")
    print(f"  Score: {scores_hyde[0]:.4f}")
    print(f"  Chapter: {CHUNKS[indices_hyde[0]].get('chapter_title', 'N/A')}")
    
    improvement = scores_hyde[0] - scores_normal[0]
    print(f"\n개선도: {improvement:+.4f} ({'↑' if improvement > 0 else '↓' if improvement < 0 else '→'})")
    
    results.append({
        'query': query,
        'normal_score': scores_normal[0],
        'hyde_score': scores_hyde[0],
        'improvement': improvement
    })

# 전체 요약
print("\n" + "=" * 80)
print("[전체 요약]")
print("=" * 80)
avg_improvement = np.mean([r['improvement'] for r in results])
print(f"평균 개선도: {avg_improvement:+.4f}")
print(f"개선된 쿼리: {sum(1 for r in results if r['improvement'] > 0)}/{len(results)}")
print("\n✅ HyDE 테스트 완료!")


---
## 7️⃣ 요약 및 다음 단계

### 학습 내용 요약

1. **HyDE 개념**: 쿼리를 가상의 답변 문서로 확장하여 검색
2. **효과**: 쿼리-문서 간 형식 차이를 해소하여 검색 정확도 향상
3. **구현**: Ollama를 활용한 로컬 LLM 기반 가상 문서 생성
4. **검증**: 동일 쿼리에 대해 일반 검색보다 높은 유사도 점수 확인

### HyDE의 장점

- ✅ 짧은 쿼리와 긴 문서 간 의미적 거리 해소
- ✅ 복잡한 질문의 의도를 더 잘 포착
- ✅ 다양한 표현 방식을 통합
- ✅ 추가 인덱싱 불필요 (런타임에만 적용)

### HyDE의 고려사항

- ⚠️  LLM 호출로 인한 레이턴시 증가
- ⚠️  가상 문서가 실제 답변과 다를 수 있음 (hallucination 가능)
- ⚠️  짧고 간단한 쿼리에는 오버헤드

### 다음 단계

- **Chapter 7.3**: Reranking (검색 결과 재정렬)
- **Chapter 7.4**: Self-RAG (자기 평가 기반 검색)
- **Chapter 7.5**: 모든 기법 통합 (CCH + HyDE + Reranking + Self-RAG)

### 주요 함수

- `generate_hypothetical_document(query)`: 쿼리로부터 가상 문서 생성
- `search_with_text(text, k)`: 텍스트를 임베딩하여 검색
